# Ligand-Receptor Inference

As a result of the growing interest in cell-cell communication (CCC) inference, a number of computational tools in single-cell transcriptomics have emerged. Although, there many different categories and approaches to infer CCC events, in this tutorial we will focus on those that infer interactions between ligands and receptors, commonly referred to as ligand-receptor (LR) inference methods (e.g. ([Efremova et al., 2020](https://www.nature.com/articles/s41596-020-0292-x), [Hou et al., 2020](https://www.nature.com/articles/s41467-020-18873-z), [Jin et al., 2021](https://www.nature.com/articles/s41467-021-21246-9), [Raredon et al., 2022](https://www.nature.com/articles/s41598-022-07959-x)). These tools typically rely on gene expression information as a proxy of protein abundance, and they work downstream of data pre-processing and acquisition of biologically-meaningful cell groups. These CCC tools infer intercellular interactions in a hypothesis-free manner, meaning that they infer all possible interactions between cell clusters, relying on prior knowledge of the potential interactions. Here, one group of cells is considered the source of the communication signal, sending a ligand, and the other is the receiver of the signal via its receptors. CCC events are thus represented as interactions between LR pairs, expressed by any combination of source and receiver cell groups.

The information about the interacting proteins is commonly extracted from prior knowledge resources. In the case of LR methods, the interactions can also be represented by heteromeric protein complexes ([Efremova et al., 2020](https://www.nature.com/articles/s41596-020-0292-x), [Jin et al., 2021](https://www.nature.com/articles/s41467-021-21246-9), [Noël et al., 2021](https://www.nature.com/articles/s41467-021-21244-x)). 

Here, we will use LIANA to obtain a consensus score for each LR interaction inferred by the different tools. Further, we will make use of LIANA's consensus resource, which combines a number of expert-curated LR resources.

## Environment Setup

In [ ]:
library(liana, quietly = TRUE)

library(Seurat, quietly = TRUE)

library(dplyr, quietly = TRUE)
library(reshape2, quietly = TRUE)
library(ggplot2, quietly = TRUE)
library(tidyr, quietly = TRUE)

## Directories

In [ ]:
data.path = '..//..//data'
output_folder = file.path(data.path, 'liana-outputs/')

## Data Reminder

Here, we will re-load the processed data as processed in Tutorial 01.

**Change this to reading directly from Seurat once tutorial 01 in R is fully completed**

In [3]:
covid_data <- readRDS(file.path(data.path, 'covid_balf_norm.rds'))

just as a quick reminder, let's visualize the cell types and samples in the data.

In [4]:
head(covid_data@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,sample,sample_new,group,disease,hasnCoV,cluster,celltype,condition,percent.mt
,<fct>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<dbl>
AAACCCACAGCTACAT_3,balf.covid,3123,1376,C100,HC3,HC,N,N,27,B,Control,9.1898815
AAACCCATCCACGGGT_3,balf.covid,1429,835,C100,HC3,HC,N,N,23,Macrophages,Control,0.9097271
AAACCCATCCCATTCG_3,balf.covid,2342,1105,C100,HC3,HC,N,N,6,T,Control,6.3193851
AAACGAACAAACAGGC_3,balf.covid,31378,4530,C100,HC3,HC,N,N,10,Macrophages,Control,9.9815157
AAACGAAGTCGCACAC_3,balf.covid,12767,3409,C100,HC3,HC,N,N,10,Macrophages,Control,5.1617451
AAACGAAGTCTATGAC_3,balf.covid,2198,1094,C100,HC3,HC,N,N,9,T,Control,8.8717015


In [ ]:
# plot pre-annotated cell types
h_ = 5
w_ = 14
options(repr.plot.height=h_, repr.plot.width=w_)

Idents(covid_data)<-'celltype'
plot1<-DimPlot(covid_data, reduction = "umap", group.by = 'cell.type')
plot2<-DimPlot(covid_data, reduction = "umap", group.by = 'sample_new')

plot1 + plot2

## Predicting CCC events with LIANA


<img src="https://github.com/saezlab/liana/blob/master/vignettes/ligrec_pipe.png?raw=true" width="900"/>


Now that we have the preprocessed data loaded, we will use liana to score the interactions inferred by the different tools.

`liana` is highly modularized and it implements a number of methods to score LR interactions, we can list those with the following command: 

In [5]:
liana::show_methods()

[1] "connectome"      "logfc"           "natmi"           "sca"            
 [5] "cellphonedb"     "cytotalk"        "call_squidpy"    "call_cellchat"  
 [9] "call_connectome" "call_sca"        "call_italk"      "call_natmi"

liana classifies interaction scores into two categories: those that infer the **Magnitude** and **Specificity** of interactions. The **Magnitude** of an interactions is a measure of the strength of the interaction's expression, and the **Specificity** of an interaction is a measure of how specific is an interaction to a given pair of clusters. Generally, these categories are complementary, and the magnitude of the interaction is a proxy of the specificity of the interaction. For example, a ligand-receptor interaction with a high magnitude score is likely to be specific, and vice versa. 

## Scoring Functions

We will now describe the mathematical formulation of the magnitude and specificity scores we will use in this tutorial:

### Shared Notations


`k` is the k-th ligand-receptor interaction 

`L` - expression of ligand L

`R` - expression of receptor R

`C` - cell cluster

`i` - cell group i

`j` - cell group j

`M` - the library-size normalized and log1p-transformed gene expression matrix

`X` - normalized gene expression vector


### CellPhoneDBv2
**Magnitude**: $$ LRmean_{k,i,j} = \frac{L_{C_{i}} + R_{C_{j}}}{2}$$

**Specificity**: CellPhoneDBv2 introduced a **permutation approach** also adapted by other methods, see permutation formulation below.


**Specificity**: CellPhoneDBv2 introduced a **permutation approach** also adapted by other methods, see permutation formulation below.



### Geometric Mean
**Magnitude**: $$ LRgeometric.mean_{k,i,j} = \sqrt{L_{C_{i}} \cdot R_{C_{j}}}$$

**Specificity**:
An adaptation of CellPhoneDBv2's permutation approach.


### CellChat (a resource-agnostic adaptation)

**Magnitude**:

$$ LRprob_{k,i,j} = \frac{TriMean(L_{C_{i}}) \cdot

TriMean(R_{C_{j}})}{Kh + TriMean(L_{C_{i}}) \cdot

TriMean(R_{C_{j}})} $$

where Kh = 0.5 by default and `TriMean` represents Tuckey's Trimean function:

$$TriMean(X) = \frac{Q_{0.25}(X) + 2 \cdot Q_{0.5}(X) + Q_{0.75}(X)}{4}$$

Note that the original CellChat implementation also uses information of mediator proteins, which is specific to the CellChat resource.
Since we can use any resource with liana, by default liana's consensus resource, we will not use this information, and hence the implementation of CellChat's `LR_probability` in LIANA was simplified to be resource-agnostic.

**Specificity**:
An adaptation of CellPhoneDBv2's permutation approach.



##### The specificity scores of these three method is calculated as follows:

$$p\text{-value}_{k,i,j} = \frac{1}{P} \sum_{p=1}^{P} [fun_{permuted}(L^*_{C_{i}}, R^*_{C_{j}}) \geq fun_{observed}(L^*_{C_{i}}, R^*_{C_{j}})]$$

where `P` is the number of permutations, and `L*` and `R*` are ligand and receptor expression summarized according by each method, i.e. `arithmetic mean` for CellPhoneDB and Geometric Mean, and `TriMean` for CellChat.


### SingleCellSignalR

**Magnitude**:

$$LRscore_{k,i,j} = \frac{\sqrt{L_{C_{i}} R_{C_{j}}}}{\sqrt{L_{C_{i}} R_{C_{j}}} + \mu}$$

where `mu` is the mean of the expression matrix `M`


### NATMI

**Magnitude**: $$LRproduct_{k,i,j} = L_{C_{i}} R_{C_{j}}$$

**Specificity**: $$SpecificityWeight_{k,i,j} = \frac{L_{C_{i}}}{\sum^{n} L_{C_{i}}} \cdot \frac{R_{C_{j}}}{\sum^{n} R_{C_{j}}}$$


### Connectome

**Magnitude**: $$LRproduct_{k,i,j} = L_{C_{i}} R_{C_{j}}$$

**Specificity**:  $$ LRz.mean_{k,i,j} = \frac{z_{L_{C_{i}}} + z_{R_{C_{j}}}}{2} $$

where `z` is the z-score of the expression matrix `M`:

$$ X_{z} = (X - mean(X)) / std(X) $$


### log2FC

**Specificity**: $$ LRlog2FC_{k,i,j} = \frac{\text{Log2FC}_{C_i,L} + \text{Log2FC}_{C_j,R}}{2} $$


where log2FC for each gene is calculated as: $$ log2FC = \log_2\left(\text{mean}(X_i)\right) - \log_2\left(\text{mean}(X_{\text{not}_i})\right) $$



What the above equations show is that there are many commonalities between the different methods, yet there are also many variations in the way the magnitude and specificity scores are calculated. 

$$ I \left\{ L_{C_j}^{expr.prop} \geq 0.1 \text{ and } R_{C_j}^{expr.prop} \geq 0.1 \right\} $$

where liana considers interactions as occurring only if the ligand and receptor, and **all** of their subunits, are expressed by default in at least 0.1 of the cells (`>= expr_prop`) in cell clusters `i`, `j`.
Any interactions that don't pass these criteria are not returned by default, to return them the user can check the `return_all_lrs` parameter.

### Score Distributions

In [ ]:
# pick a sample to infer the communication scores for
sdata = subset(x = covid_data, subset = sample == 'C100')
sdata

While LIANA can work with Seurat objects, it has more functionality with SingleCellExperiment objects. We can easily convert a Seurat object to a SingleCellObject:

In [ ]:
sdata <- Seurat::as.SingleCellExperiment(sdata)
sdata

The parameters that we will use are the following:
- `idents_col` is the column in the `meta` dataframe that contains the cell groups
- `assay` is a string that indicates which Seurat assay to use (typically `RNA`, unless having done a batch correction step)
- `assay.type` is a string that indicates whether to use the raw (`counts`) or log- and library-normalized (`logcounts`) counts attribute of the Seurat object assay. Since most CCC tools expect library- and log-normalized data, we will use that. 
- `expr_prop` is the proportion of cells that need to express a ligand-receptor pair for it to be considered as a ligand-receptor pair
- `min_cells` is the minimum number of cells per cell type required for a cell type to be considered in the analysis
- `verbose` is a boolean that indicates whether to print the progress of the function

(Other parameters are described in the documentation of the function, as well as in more detail below)

In [ ]:
liana_res <- liana_wrap(sce = sdata, idents_col='celltype', assay.type = 'logcounts', min_cells=5,                      
                        expr_prop=0.1, verbose=TRUE # , parallelize = TRUE, workers = 30 # not everyone would have 30 workers :)
                      )

liana returns a list of results, each element of which corresponds to a method

In [6]:
liana_res %>% dplyr::glimpse()

ERROR: Error in dplyr::glimpse(.): object 'liana_res' not found


### LIANA's Rank Aggregate

LIANA can calculate an aggregate rank for both `magnitude` and `specificity` as defined above. The `rank_aggregate` function of liana uses a re-implementation of the RobustRankAggregate method by [Kolde et al., 2012](https://pubmed.ncbi.nlm.nih.gov/22247279/), and generates a probability distribution for ligand-receptors that are ranked consistently better than expected under a null hypothesis. It thus provides a consensus of the rank of the ligand-receptor interactions across methods, that can also be treated as a p-value.

In more detail, a rank aggregate is calculated for the `magnitude` and `specificity` scores from the methods in LIANA as follows:

First, a normalized rank matrix[0,1] is generated separately for magnitude and specificity as: $$ r_{i,j} = \frac{rank_{i,j}}{\max(rank_i)} \quad (1 \leq i \leq m, 1 \leq j \leq n) $$


where `m` is the number of score rank vectors, `n` is the length of each score vector (number of interactions), **rank<sub>i,j</sub>** is the rank of the `j-th` element (interaction) in the `i-th` score rank vector, and **max(rank<sub>i</sub>)** is the maximum rank in the `i-th` rank vector.

For each normalized rank vector `r`, we then ask how probable is it to obtain **r<sup>null</sup><sub>(k)</sub> <= r<sub>(k)</sub>**, where **r<sup>null</sup><sub>(k)</sub>** is a rank vector generated under the null hypothesis. The RobustRankAggregate method expresses [Kolde et al., 2012](https://pubmed.ncbi.nlm.nih.gov/22247279/) the probability **r<sup>null</sup><sub>(k)</sub> <= r<sub>(k)</sub>** as **β<sub>k,n</sub>(r)**, through a beta distribution. 
This entails that we obtain probabilities for each score vector `r` as: $$ p(r) = \underset{1, ..., n}{min} \beta_k,_n(r) * n $$



where we take the minimum probability `ρ` for each interaction across the score vectors, and we apply a Bonferroni correction to the p-values by multiplying them by `n` to account for multiple testing.

Aggregate scores are also non-negative, which is beneficial for decomposition with Tensor-cell2cell.

Calculate both the magnitude and specificity consensus rank scores:

In [ ]:
liana_agg <- rank_aggregate(liana_res = liana_res)

Here, for each scoring type of specificity and magnitude, we can see the different scores for each method as well as the consensus scores. Consensus scores are delineated by the `specificity_rank` and `magnitude_rank` columns.

In [ ]:
head(liana_agg)

In [ ]:
# Pivot to a long format
scores_long <- liana_agg %>%
    pivot_longer(cols = -c('source', 'target', 'ligand.complex', 'receptor.complex'),
    names_to = 'score', values_to = 'value')

In [ ]:
head(scores_long)

For clarity, here we map each output score (column names in the above dataframe) to the scoring method and scoring type:

| Method Name | Magnitude Score | Specificity Score |
| :- | -: | :-: |
| CellPhoneDB | lr.mean | cellphonedb.pvalue
| Connectome | prod_weight | weight_sc
| log2FC | None | logfc_comb
| NATMI | prod_weight | edge_specificity
| SingleCellSignalR (sca) | LRscore | None

### Score Distributions

To provide a better illustration of the different scores, we will now plot the distribution of scores for each of the methods.

In [ ]:
h_ = 7
w_ = 14
options(repr.plot.height=h_, repr.plot.width=w_)

# keep only method scores
scores_long <- scores_long %>% filter(!stringr::str_detect(score, "_rank"))

ggplot(scores_long, aes(x = value, fill = score)) +
    geom_density(alpha = 0.5) + 
    facet_wrap('~score', scales='free', ncol = 4) +
    theme_bw()

This show the extent to which  each of the scoring functions differ from each other. Unsurprisingly, independent evaluations have shown that the choice of method and/or resource leads to limited consensus in inferred predictions when using different tools ([Dimitrov et al., 2022](https://www.nature.com/articles/s41467-022-30755-0), [Liu et al., 2022](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-022-02783-y), [Wang et al., 2022](https://academic.oup.com/bfg/article/21/5/339/6640320)). This is why we have implemented a consensus score that is a rank aggregate of the scores obtained by the different methods, and for the sake of these tutorials we will use this consensus score to rank the interactions, specifically the one that aggregates the magnitude scores from different functions. Aggregate scores are also non-negative, which is beneficial for decomposition with Tensor-cell2cell.

### Single-Sample Dotplot

Let's generate a basic dotplot with the most highly-ranked ligand-receptor interactions for this sample from LIANA's aggregate scores. 

In [ ]:
liana_agg_int <- liana_agg %>%
  # only keep interactions with p-val <= 0.05
  filter(specificity_rank <= 0.05) %>% 
  distinct_at(c("ligand.complex", "receptor.complex")) 

In [ ]:

# keep only those interactions that are significant in at least one cell pair, and rank to magnitude
liana_agg <- liana_agg %>%
  # keep only the interactions of interest
  inner_join(liana_agg_int, 
             by = c("ligand.complex", "receptor.complex")) %>%
  # then rank according to `magnitude`
  arrange(magnitude_rank)

Since both lower specificity and lower magnitude consensus rank scores have higher communication importance, we set the invert parameters to TRUE:

In [ ]:
h_ = 10
w_ = 17
options(repr.plot.height=h_, repr.plot.width=w_)

liana_dotplot(
  liana_agg,
  source_groups = c('B', 'Macrophages', 'pDC'),
  target_groups = c('Mast', 'NK', 'T', 'pDC'),
  ntop = 20,
  specificity = "specificity_rank",
  magnitude = "magnitude_rank",
  invert_magnitude = TRUE, 
  invert_specificity = TRUE
)

Great! We have now obtained ligand-receptor predictions for a single sample. What we see here is that interactions are predicted across cell types and are typically specific to pairs of cell types.

Note that missing dots here would represent interactions for which the ligand and receptor are not expressed above the `expr_prop`.

## Run LIANA by Sample

Now that we have familiarized ourselves with how ligand-receptor methods in LIANA work and how the results look by sample, let's run LIANA on all of the samples in the dataset. These results will be used to generate a tensor of ligand-receptor interactions across contexts that will be decomposed into CCC patterns by `Tensor-Cell2cell`.

This is easily done with liana with the `liana_bysample` function. This function takes the same inputs was `liana_wrap`, but it will also iterate across samples as specified in the metadata. It also automatically aggregates the scores by specifying `aggregate_how`.

We relax the `expr_prop` parameter to have more consistent presence of LR interactions across contexts. See further discussions on the `how` parameter in Tutorial 03. 

Results are stored in the SCE object as `sce@metadata$liana_res`

In [ ]:
covid_data <- Seurat::as.SingleCellExperiment(covid_data)

In [ ]:
covid_data <- liana_bysample(sce = covid_data,
                      sample_col = "sample_new", # context dimension column from SCE metadata
                      idents_col = "celltype", # cell types column from SCE metadata
                      expr_prop = 0.1, 
                      assay.type = 'logcounts',
                      aggregate_how = 'both',
                      verbose = TRUE,
                      permutation.params=list(nperm=3)
                      # parallelize = TRUE, workers = 30
                      )

### Check the results

Here, we can combine the results and see that they look very similar as before with the exception that we have an additional column for the sample name, and the results are now by sample.

In [ ]:
liana_res <- covid_data@metadata$liana_res %>% 
            bind_rows(.id = "sample_new")

We can also generate a DotPlot for each sample.
Let's pick the first two distinct interaction in the list, and see what they look like.

In [ ]:
ligand_complex <- 'B2M'
receptor_complex <- c('CD3D', 'KLRD1')
source_labels <- c("B", "pDC", "Macrophages")
target_labels <- c("T", "Mast", "pDC", "NK")
sample_key <- 'sample_new'
colour <- 'magnitude_rank'
size <- "specificity_rank"

Filter to interactions of interest, and invert the specificity and magnitude ranks to have higher scores indicate higher communication importance.

In [ ]:
liana_res <- liana_res %>% 
    filter(source %in% source_labels) %>%
    filter(target %in% target_labels) %>%
    filter(ligand.complex == ligand_complex) %>%
    filter(receptor.complex %in% receptor_complex) %>%
    unite(lr.pairs, ligand.complex, receptor.complex, sep = ' -> ') %>%
    # inverse ranks
    mutate_at(vars(specificity_rank, magnitude_rank), ~-log10(. + 1e-10))

In [ ]:
ggplot(lv, aes(x = target, y = source, color = .data[[colour]], size=.data[[size]])) + geom_point() + 
facet_wrap(lr.pairs ~ sample_new , ncol = 12) + 
theme_bw()

Here, we can already see that the ligand-receptor interactions are not only specific to cell types, but also to samples or contexts (see **B2M -> KLRD1** in samples *M1*, *M2*, *M3*). However, we can also see that this plot, even with just two LR interactions visualized, starts to get a bit overwhelming. To this end, to make the most use of the hypothesis-free nature of the ligand-receptor interactions, in the next chapter we will use `Tensor-Cell2cell` to decompose the ligand-receptor interactions into interpretable CCC patterns across contexts.

Save Results for Tensor-Cell2cell.

In [ ]:
saveRDS(covid_data.sce@metadata$liana_res, file.path(output_folder, 'LIANA_by_sample_R.rds'))

## Supplementary Information about LIANA

### Key Parameters

We already covered some of the parameters in LIANA, that can be used to customize the results. Here, we will go in more detail over some of the most important ones.


- `resource` and `resource_name` enable the user to select the resource that they want to use for CCC inference. By default, liana will use the 'consensus' resource which combines a number of expert-curated ligand-receptor resources. However, one can also use the 'cellphonedb' resource, any of the resources that are available within liana by passing their `resource_name` (See below). Additionally, the user can pass their own resource via the `resource` parameter, which expects a pandas DataFrame.


In [ ]:
show_resources()


- `expr_prop` which we also used before is the proportion of cells that need to express a ligand-receptor pair for it to be considered as a potential ligand-receptor pair. This is a parameter that can be used to filter out lowly-expressed ligand-receptor pairs. This is common practice in CCC inference at the cluster level, as we make the assumption that the event occurs for all cells within that clusters.

- `return_all_lrs` is related to the `expr_prop` parameter. If `return_all_lrs` is set to `True`, then all ligand-receptor pairs will be returned, regardless of whether they are expressed above the `expr_prop` threshold. This is useful if one wants to use the `expr_prop` parameter to filter out lowly-expressed ligand-receptor pairs, but still wants to see the scores for all ligand-receptor pairs.

In addition to those parameters liana provides a number of other utility parameters as well as some method-specific ones, please refer to the documentation for more information.

## Working with Seurat objects directly

####  <span style='color:red'> ^Daniel's Note: we can just import a seurat object and run liana on it. (and do the rest in SingleCellExperiment)

To run on one sample is the same syntax as using a SCE object:

In [ ]:
liana_res <- liana_wrap(sce = sdata.so, 
                        idents_col='cell.type', 
                        assay='RNA', # specify the Seurat assay
                        assay.type = 'logcounts', 
                        expr_prop=0.1, 
                        verbose=T, 
                        parallelize = T, 
                        workers = 30)

Note, by default, LIANA runs all the available scoring methods. You can select only certain methods to run by specifiying the `method` argument. For example, if I only wanted to run CellPhoneDB and NATMI, I could specify it has follows:

In [ ]:
liana_res <- liana_wrap(sce = sdata.so, 
                        idents_col='cell.type', 
                        assay='RNA', # specify the Seurat assay
                        assay.type = 'logcounts', 
                        expr_prop=0.1,
                        method = c('cellphonedb', 'natmi') # only two scoring methods
                        verbose=T, 
                        parallelize = T, 
                        workers = 30)

Additionally, if you want to just rank by the magnitude or the specificity, rather than both, you can use the following. Here, since we filtered for only CellPhoneDB and NATMI in the cell above, this will only calculate a consensus score for those two methods. Since both these methods return a magnitude and specifity score, a consensus score can be calculated for each score type. 

In [ ]:
liana_aggregate.magnitude <- liana_aggregate(liana_res = liana_res, aggregate_how='magnitude', verbose = F)
liana_aggregate.specificity <- liana_aggregate(liana_res = liana_res, aggregate_how='specificity', verbose = F)

`liana_bysample` does not work directly on Seurat objects. However, we can manually iterate through samples to create a `context_df_dict` dictionary (named list) which can be used as input to the `liana_tensor_c2c` function used to build the tensor in Tutorial 03:

In [ ]:
context_df_dict<-list()
for (sample.name in sort(unique(covid_data$sample_new))){
    sdata_ = subset(x = covid_data, subset = sample_new == sample.name)
    liana_res_ <- liana_wrap(sce = sdata_, idents_col='cell.type', assay='RNA', assay.type = 'logcounts', 
                          expr_prop=0.05, verbose=F, parallelize = T, workers = 30)

    liana_aggregate.magnitude_ <- liana_aggregate(liana_res = liana_res_, aggregate_how='magnitude', verbose = F)
    
    # retain only the aggregate magnitude rank score
    # and format for input to liana_tensor_c2c function
    liana_aggregate.magnitude_<-liana_aggregate.magnitude_[1:5]
    colnames(liana_aggregate.magnitude_)<-c('source', 'target', 'ligand.complex', 'receptor.complex', 'magnitude_rank')
    liana_aggregate.magnitude_[['sample_new']]<-sample.name
    
    context_df_dict[[sample.name]]<-liana_aggregate.magnitude_
}

The output of this function can directly be input to the `liana_tensor_c2c` function used in Tutorial 03. For more details, see liana's tensor-cell2cell [tutorial](https://saezlab.github.io/liana/articles/liana_cc2tensor.html). 

### Rank Comparisons

**Probably want to remove this**
Let's see how the specificty and magnitude aggregate scores compare using a Spearman correlation:

In [ ]:
cor.test(lav$magnitude_rank, lav$specificity_rank, method = c('spearman'))$estimate[['rho']]

#### Comparison with Python output:

There are minor differences with the LIANA implementation in python that lead to outputs not being identical

- SingleCellSignalR Magnitude (LRscore): slightly different after 3rd decimal place
- LogFC Specificity (logfc_comb): similar relative differences but different exact values
- CellPhoneDB Specificity (pvalue): similar relative differences but different exact values
- CellChat: not run by default in R

To have more consistency with Python between, for example, the magnitude aggregate rank score, we can run LIANA using the two methods that are 100% consistent between the two languages (CellPhoneDB and NATMI) and re-calculate our magnitude score from just those two. 

In [ ]:
liana_res_partial <- liana_wrap(sce = sdata, idents_col='cell.type', assay='RNA', assay.type = 'logcounts', 
                      expr_prop=0.1, verbose=T, parallelize = T, workers = 30, 
                      method = c("natmi", "cellphonedb"))
liana_aggregate_partial <- liana_aggregate(liana_res = liana_res_partial, 
                                           aggregate_how='magnitude', verbose = F)
liana_aggregate_partial<-liana_aggregate_partial[c('source', 'target', 'ligand.complex', 'receptor.complex', 'aggregate_rank')]

head(liana_aggregate_partial)

Because the rank aggregation **insert explanation here**, the aggregated magnitude rank scores slightly differ in absolute value. However, they agree highly in terms of relative values, as seen by the Spearman correlation:

In [ ]:
liana_aggregate_partial <- plyr::arrange(liana_aggregate_partial, source, target, ligand.complex, receptor.complex)
write.csv(liana_aggregate_partial, paste0(output_folder, 'magnitude_ranks_R.csv'))

# read and format python aggregate rank
lap_python<-read.csv(paste0(output_folder, 'magnitude_ranks_python.csv'))
clp<-colnames(lap_python)
clp[4]<-'ligand.complex'
clp[5]<-'receptor.complex'
colnames(lap_python)<-clp
lap_python<-lap_python[colnames(lap_python) != 'X']

# merge the two scores
la<-dplyr::inner_join(liana_aggregate_partial, lap_python)

spearmanr<-cor.test(la$aggregate_rank, 
         la$magnitude_rank, method = c('spearman'))$estimate[['rho']]
print(paste0('The spearman correlation is: ', spearmanr))

In [ ]:
liana_aggregate.magnitude <- plyr::arrange(liana_aggregate.magnitude, source, target, ligand.complex, receptor.complex)
write.csv(liana_aggregate.magnitude, file.path(output_folder, 'magnitude_ranks_all_R.csv'))
